In [3]:
# =============================================================================
# 🦟 NATURAL-LOOKING INSECT BITE AUGMENTATION (Refined Realistic Version)
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

!pip install -q albumentations==1.4.3 opencv-python Pillow scikit-image
!pip install --force-reinstall -q numpy==1.24.3

import cv2
import numpy as np
import os
import random
import albumentations as A
from PIL import Image, ImageEnhance
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries installed successfully!")

# =========================
# PATH SETUP
# =========================
dataset_path = '/content/drive/My Drive/Dataset/Real_Images/Train'
output_path = '/content/drive/My Drive/Dataset/augmented_realistic'
os.makedirs(output_path, exist_ok=True)

print(f"📂 Input path: {dataset_path}")
print(f"💾 Output path: {output_path}")

# =========================
# IMAGE ENHANCEMENT (subtle + clean)
# =========================
def enhance_image_quality(image):
    """Light enhancement to maintain realistic skin tone and bite visibility"""
    try:
        pil_img = Image.fromarray(image)

        # Subtle sharpness + clarity
        pil_img = ImageEnhance.Sharpness(pil_img).enhance(1.1)

        # Gentle contrast
        pil_img = ImageEnhance.Contrast(pil_img).enhance(1.1)

        # Slightly boost natural colors
        pil_img = ImageEnhance.Color(pil_img).enhance(1.03)

        enhanced = np.array(pil_img)

        # Gentle histogram normalization (avoid over-contrast)
        ycrcb = cv2.cvtColor(enhanced, cv2.COLOR_RGB2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        enhanced = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2RGB)

        # Mild denoising (retains texture)
        enhanced = cv2.fastNlMeansDenoisingColored(enhanced, None, 3, 3, 5, 15)

        return enhanced
    except Exception as e:
        print(f"❌ Enhancement error: {e}")
        return image

# =========================
# REALISTIC AUGMENTATION PIPELINE
# =========================
def create_realistic_pipelines():
    """Augmentations that mimic natural photo variability"""
    return A.Compose([
        # Light brightness/contrast variation (like camera lighting)
        A.RandomBrightnessContrast(brightness_limit=0.12, contrast_limit=0.12, p=0.7),

        # Slight adaptive histogram enhancement (gentle local contrast)
        A.CLAHE(clip_limit=1.5, tile_grid_size=(8, 8), p=0.3),

        # Random rotations & flips (minor)
        A.Rotate(limit=12, p=0.6),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.15),

        # Mild shift/zoom/scale (preserves lesion shape)
        A.ShiftScaleRotate(
            shift_limit=0.03,
            scale_limit=0.08,
            rotate_limit=10,
            border_mode=cv2.BORDER_REFLECT_101,
            p=0.7
        ),

        # Soft blur or noise (as if from camera)
        A.OneOf([
            A.GaussianBlur(blur_limit=(3, 5), p=0.3),
            A.MedianBlur(blur_limit=3, p=0.2),
            A.GaussNoise(var_limit=(5, 10), mean=0, p=0.3),
        ], p=0.5),

        # Mild RGB shift (within skin tone tolerance)
        A.RGBShift(r_shift_limit=8, g_shift_limit=8, b_shift_limit=8, p=0.15),

        # Subtle hue/saturation change (preserves natural color)
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.2),

        # Gentle CoarseDropout (to simulate imperfect lighting)
        A.CoarseDropout(max_holes=4, max_height=8, max_width=8, fill_value=None, p=0.15)
    ])

# =========================
# AUGMENTATION FUNCTION
# =========================
def augment_and_save_class(class_name, target_per_class=300):
    """Augment a single class with realistic transformations"""
    class_path = os.path.join(dataset_path, class_name)
    output_class_path = os.path.join(output_path, class_name)
    os.makedirs(output_class_path, exist_ok=True)

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    if not images:
        print(f"❌ No images found in {class_name}")
        return

    print(f"\n📸 Processing {class_name} — Found {len(images)} originals")

    originals = []
    for filename in images:
        path = os.path.join(class_path, filename)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            originals.append(img)

    aug_pipeline = create_realistic_pipelines()
    all_augmented = []

    while len(all_augmented) < target_per_class:
        img = random.choice(originals)
        enhanced = enhance_image_quality(img)
        augmented = aug_pipeline(image=enhanced)['image']
        all_augmented.append(augmented)

    print(f"💾 Saving {len(all_augmented)} augmented images for {class_name}...")

    for i, img in enumerate(all_augmented):
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        filename = f"augmented_realistic_{i:04d}.jpg"
        cv2.imwrite(os.path.join(output_class_path, filename), img_bgr)

    print(f"✅ Done: {class_name} — {len(all_augmented)} images saved")

# =========================
# RUN AUGMENTATION FOR ALL CLASSES
# =========================
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

for cls in classes:
    augment_and_save_class(cls, target_per_class=300)

print("\n🎯 All classes processed successfully!")
print(f"💾 Saved realistic augmentations to: {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
✅ Libraries installed successfully!
📂 Input path: /content/drive/My Drive/Dataset/Real_Images/Train
💾 Output path: /content/drive/My Drive/Dataset/augmented_realistic

📸 Processing bed_bugs_bite — Found 199 originals
💾 Saving 300 au